In [1]:
# %tensorflow_version 2.x
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)
# GD = "/content/drive/My Drive/DeepLearning/"
# import sys
# sys.path.append(GD + 'code/')

In [1]:
import numpy as np
import scipy as sp

import os

import warnings
warnings.filterwarnings('ignore')
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

import tensorflow.keras as keras
import tensorflow as tf

import pickle
import random

from sklearn.model_selection import KFold
from scipy.sparse import block_diag as sparse_block_diag
from sklearn.preprocessing import LabelEncoder

from datetime import datetime

from utils.labels_preprocessing import load_labels
from utils.load_synced_inputs import load_inputs, load_fmri
from utils.load_graphs import load_graph
from cnn_graph.resnetgraph import cgcnn
from cnn_graph import graph
import tensorflow.keras.backend as K
from new_utils import utils

In [2]:
tf.test.is_gpu_available()

True

In [3]:
# fmri preprocess

fmri_subcortical_dir = '../Subcortical_fmri'
# fmri_subcortical_dir = 'C:/Users/YuNan/Downloads/auto_encoder_try/data/Subcortical_fmri'
assert len(os.listdir(fmri_subcortical_dir)) == len(set(os.listdir(fmri_subcortical_dir)))


In [4]:
home = f"{os.getcwd()}/.."

graphs_dir = "../Graphs/"            # directory containing necessary graphs
# print(graphs_dir)
# cortical_dir = f"{home}/Cortical"         # cortical node data for each subject
# subcortical_dir = f"{home}/SubCortical"   # subcortical node data for each subject
# cortical_dir = 'C:/Users/YuNan/Downloads/auto_encoder_try/data/Cortex'
subcortical_dir = '../Subcortical'

# for scan in os.listdir(cortical_dir):
#     print (f"{scan[:-11]}_Subcortical")
#     assert os.path.isfile(f"{subcortical_dir}/{scan[:-11]}_Subcortical.mat")

assert len(os.listdir(subcortical_dir)) == len(set(os.listdir(subcortical_dir)))

# L = {'cortical': load_graph(graphs_folder=graphs_dir, prefix='M_w')}
# # print(L)
# L['cortical'] = [graph.laplacian(A) for A in L['cortical']]
# # print(L['cortical'])

L = {'subcortical' : load_graph(graphs_folder=graphs_dir, prefix='M_sc_w')}
L['subcortical'] = [graph.laplacian(A) for A in L['subcortical']]
print(L['subcortical'])

# # Block Diagonalize the Cortical & Subcortical Laplacians at Each Level of Coarsening
new_L = []
for A in zip(L[f"subcortical"]):
    new_L.append( sparse_block_diag((A)) )
L = new_L
# print(L)
del new_L

[<14848x14848 sparse matrix of type '<class 'numpy.float32'>'
	with 103768 stored elements in Compressed Sparse Row format>, <7424x7424 sparse matrix of type '<class 'numpy.float32'>'
	with 51800 stored elements in Compressed Sparse Row format>, <3712x3712 sparse matrix of type '<class 'numpy.float32'>'
	with 25816 stored elements in Compressed Sparse Row format>, <1856x1856 sparse matrix of type '<class 'numpy.float32'>'
	with 12824 stored elements in Compressed Sparse Row format>, <928x928 sparse matrix of type '<class 'numpy.float32'>'
	with 6328 stored elements in Compressed Sparse Row format>, <464x464 sparse matrix of type '<class 'numpy.float32'>'
	with 3080 stored elements in Compressed Sparse Row format>, <232x232 sparse matrix of type '<class 'numpy.float32'>'
	with 1456 stored elements in Compressed Sparse Row format>, <116x116 sparse matrix of type '<class 'numpy.float32'>'
	with 648 stored elements in Compressed Sparse Row format>]


In [5]:
L

[<14848x14848 sparse matrix of type '<class 'numpy.float32'>'
 	with 103768 stored elements in Compressed Sparse Row format>,
 <7424x7424 sparse matrix of type '<class 'numpy.float32'>'
 	with 51800 stored elements in Compressed Sparse Row format>,
 <3712x3712 sparse matrix of type '<class 'numpy.float32'>'
 	with 25816 stored elements in Compressed Sparse Row format>,
 <1856x1856 sparse matrix of type '<class 'numpy.float32'>'
 	with 12824 stored elements in Compressed Sparse Row format>,
 <928x928 sparse matrix of type '<class 'numpy.float32'>'
 	with 6328 stored elements in Compressed Sparse Row format>,
 <464x464 sparse matrix of type '<class 'numpy.float32'>'
 	with 3080 stored elements in Compressed Sparse Row format>,
 <232x232 sparse matrix of type '<class 'numpy.float32'>'
 	with 1456 stored elements in Compressed Sparse Row format>,
 <116x116 sparse matrix of type '<class 'numpy.float32'>'
 	with 648 stored elements in Compressed Sparse Row format>]

In [6]:
# load the dataset 
import pandas as pd
Path = '../data.csv'
df = pd.read_csv(Path)
filenames = df['Subject'].astype('str').to_numpy()
# print(filenames)
filenames2 = (filenames + '_Subcortical' +'.mat').tolist()
# print(filenames2)
df

,Subject
0,1000102_20252_2_0
1,1000710_20252_2_0
2,1000742_20252_2_0
3,1001275_20252_2_0
4,1001693_20252_2_0
...,...
190,1030712_20252_2_0
191,1030841_20252_2_0
192,1030960_20252_2_0
193,1031057_20252_2_0


In [9]:
# fmri 
graph_type = {'subcortical':{'path':fmri_subcortical_dir, 
                             'halves':False}
}


X = load_fmri(graph_type=graph_type, filenames = filenames2)

sub_graphs = list(graph_type.keys())
assert sub_graphs == ['subcortical']

Subcortical	(Samples, Nodes, Feat.):	(195, 14848, 490)


In [10]:
# try
X_try = {}
X_try['subcortical'] = X['subcortical'][:100,:,10:]
del X
# X_try['subcortical'] = X['subcortical'][:,:,0:200]
X_try['subcortical'].shape
X = X_try
del X_try

In [11]:
X['subcortical'].shape

(100, 14848, 480)

In [12]:
for i in range(X['subcortical'].shape[0]):
    nodes = X['subcortical'][i].flatten()
    X_mean = np.mean(nodes)
    X_std = np.std(nodes)
    X['subcortical'][i] = (X['subcortical'][i] - X_mean) / X_std
    
del nodes   
all_nodes = np.concatenate([X[sub].flatten() for sub in sub_graphs])
del all_nodes

new_X = []
for A in zip(X[f"subcortical"]):
    new_X.append(np.concatenate((A), axis = 0))
    
X = np.concatenate([np.expand_dims(i, axis=0) for i in new_X], axis=0).astype('float32')
X.shape

(100, 14848, 480)

In [14]:
# split with time point
new_X = []
# bb.T.reshape()
for i in range(X.shape[0]):
    new_X.append(X[i,:,:].T)
new_XX = np.concatenate([np.expand_dims(i, axis=0) for i in new_X], axis=0).astype('float32')
X_re = np.reshape(new_XX, (X.shape[0]* X.shape[2], X.shape[1], 1))
del X
X_re.shape

(48000, 14848, 1)

In [15]:
del new_XX
X_re2 = np.reshape(X_re, (8000,6,14848,1))

In [16]:
X_re2.shape

(8000, 6, 14848, 1)

In [19]:

X_train, X_test = X_re2[:8*800,:,:,:], X_re2[8*800:,:,:,:]
# X_ttest = X_re[39000:,:,:]
del X_re
del X_re2
print(X_train.shape, X_test.shape)
batch_size = 4

train_generator = utils.DataGenerator2(X_train, 
                                      batch_size = batch_size, 
                                      shuffle=True, 
                                      force_balance=False, 
                                      do_augmentation = False, 
#                                       tta=10, 
#                                       aug_params=aug_training,
                                      auto_encoder=True)
test_generator  = utils.DataGenerator2(X_test,  
                                      batch_size = batch_size, 
                                      shuffle=False, 
                                      force_balance=False, 
                                      do_augmentation = False, 
#                                       tta=10, 
#                                       aug_params=aug_testing,
                                      auto_encoder=True)

# ttest_generator  = utils.DataGenerator(X_ttest,  
#                                       batch_size = 1, 
#                                       shuffle=False, 
#                                       force_balance=False, 
#                                       do_augmentation = False, 
# #                                       tta=10, 
# #                                       aug_params=aug_testing,
#                                       auto_encoder=True)

(6400, 6, 14848, 1) (1600, 6, 14848, 1)


In [20]:
print(len(train_generator))
len(test_generator)

1600


400

## Architecture of the network

In [21]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mse
from tensorflow.keras import layers
from tensorflow.keras.layers import (
    Input, Dense, GRU, GlobalAveragePooling1D, 
    GlobalAveragePooling2D, TimeDistributed, 
    concatenate, Masking, Bidirectional
)
encoder_inputs = keras.Input(shape=(6,14848,1))
latent_dim = 256
Depth = 7
n_filter = 32

for B in range(Depth):
    for F in range(2):
        if B == 0 and F == 0:
            x1 = TimeDistributed(utils.ChebLayer(L[B],3,n_filter,use_bias=True,kernel_regularizer=keras.regularizers.l2(5e-5)))(encoder_inputs)
        else:
            x1 = TimeDistributed(utils.ChebLayer(L[B],3,n_filter,use_bias=True,kernel_regularizer=keras.regularizers.l2(5e-5)))(x1)
            x1 = TimeDistributed(keras.layers.BatchNormalization())(x1)
            x1 = TimeDistributed(keras.layers.Activation('elu'))(x1)
            x1 = TimeDistributed(utils.ChebLayer(L[B],3,n_filter,use_bias=False,kernel_regularizer=keras.regularizers.l2(5e-5)))(x1)
            x1 = TimeDistributed(keras.layers.BatchNormalization())(x1)
            x1 = TimeDistributed(keras.layers.Activation('elu'))(x1)
    x1 = TimeDistributed(keras.layers.MaxPooling1D(2))(x1)
x =  TimeDistributed(layers.Flatten())(x1)
        
x = LSTM(units=256, activation='elu', kernel_initializer='glorot_uniform', unit_forget_bias=True, dropout=0.3, return_sequences = True)(x)
z_mean = TimeDistributed(layers.Dense(latent_dim, name = 'z_mean'))(x)
z_log_var = TimeDistributed(layers.Dense(latent_dim, name = 'z_log_var'))(x)
#         print(z_mean.shape)
#         print(z_log_var[2][2])
        
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=K.shape(z_mean), mean=0., stddev=1.) 
    return z_mean + K.exp(z_log_var/2)*epsilon 
z = layers.Lambda(sampling, name='encoder_output')([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()


# Decoder ResNet Blocks
latent_inputs = keras.Input(shape=(6,latent_dim,))
x = TimeDistributed(layers.Dense(3712))(latent_inputs)
x = TimeDistributed(layers.Reshape((116, 32)))(x)
# print(x.shape)
for B in range(Depth):
    for F in range(2):
        #print(L[Depth - B].shape)
        if B == 0 and F == 0:
            x1 = TimeDistributed(utils.ChebLayer(L[Depth - B],3,n_filter,use_bias=False,kernel_regularizer=keras.regularizers.l2(5e-5)))(x)
        else:
            x1 = TimeDistributed(utils.ChebLayer(L[Depth - B],3,n_filter,use_bias=False,kernel_regularizer=keras.regularizers.l2(5e-5)))(x1)
        x1 = TimeDistributed(keras.layers.BatchNormalization())(x1)
        x1 = TimeDistributed(keras.layers.Activation('elu'))(x1)
        x1 = TimeDistributed(utils.ChebLayer(L[Depth - B],3,n_filter,use_bias=False,kernel_regularizer=keras.regularizers.l2(5e-5)))(x1)
        x1 = TimeDistributed(keras.layers.BatchNormalization())(x1)
        x1 = TimeDistributed(keras.layers.Activation('elu'))(x1)
    x1 = TimeDistributed(keras.layers.UpSampling1D(size=2))(x1)
decoder_outputs = TimeDistributed(utils.ChebLayer(L[0],3,1,use_bias=False,kernel_regularizer=keras.regularizers.l2(5e-5)))(x1)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

# Instantiate VAE
vae_outputs = decoder(encoder(encoder_inputs)[2])
vae         = Model(encoder_inputs, vae_outputs, name='vae')
vae.summary()



Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 6, 14848, 1) 0                                            
__________________________________________________________________________________________________
time_distributed (TimeDistribut (None, 6, 14848, 32) 128         input_1[0][0]                    
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 6, 14848, 32) 3104        time_distributed[0][0]           
__________________________________________________________________________________________________
time_distributed_2 (TimeDistrib (None, 6, 14848, 32) 128         time_distributed_1[0][0]         
____________________________________________________________________________________________

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 6, 256)]          0         
_________________________________________________________________
time_distributed_89 (TimeDis (None, 6, 3712)           953984    
_________________________________________________________________
time_distributed_90 (TimeDis (None, 6, 116, 32)        0         
_________________________________________________________________
time_distributed_91 (TimeDis (None, 6, 116, 32)        3072      
_________________________________________________________________
time_distributed_92 (TimeDis (None, 6, 116, 32)        128       
_________________________________________________________________
time_distributed_93 (TimeDis (None, 6, 116, 32)        0         
_________________________________________________________________
time_distributed_94 (TimeDis (None, 6, 116, 32)        3072

Model: "vae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 6, 14848, 1)]     0         
_________________________________________________________________
encoder (Model)              [(None, 6, 256), (None, 6 4279584   
_________________________________________________________________
decoder (Model)              (None, 6, 14848, 1)       1043680   
Total params: 5,323,264
Trainable params: 5,319,808
Non-trainable params: 3,456
_________________________________________________________________


In [22]:
#     BN_0 = keras.layers.BatchNormalization(name='BN_0')
#     x1 = BN_0(x1)
    #Layer_0  = utils.ChebLayer(L[0],4,16,0,use_bias=False,kernel_regularizer=keras.regularizers.l2(1e-12), name="Percep_0")
fmri_input = keras.layers.Input(shape=(encoder_inputs.shape[1], encoder_inputs.shape[2]))
x1 = keras.layers.BatchNormalization(name='BN_0')(fmri_input)
x1 = keras.layers.Conv1D(128, 3, padding='same', use_bias=False, kernel_initializer='glorot_uniform',
                         kernel_regularizer=keras.regularizers.l2(1e-5), name='Percep_0')(x1)
x11 = keras.layers.LeakyReLU()(x1)

x11 = keras.layers.Conv1D(128, 3, padding='same', use_bias=False, kernel_initializer='glorot_uniform',
                         kernel_regularizer=keras.regularizers.l2(1e-5), name='Percep_1')(x11)
x11 = keras.layers.LeakyReLU()(x11)
x1  = keras.layers.add([x1, -x11])
x1  = keras.layers.MaxPooling1D()(x1)

x1 = keras.layers.Conv1D(128, 3, padding='same', use_bias=False, kernel_initializer='glorot_uniform',
                         kernel_regularizer=keras.regularizers.l2(1e-5), name='Percep_2')(x1)
x11 = keras.layers.LeakyReLU()(x1)

x11 = keras.layers.Conv1D(128, 3, padding='same', use_bias=False, kernel_initializer='glorot_uniform',
                         kernel_regularizer=keras.regularizers.l2(1e-5), name='Percep_3')(x11)
x11 = keras.layers.LeakyReLU()(x11)
x1  = keras.layers.add([x1, -x11])
per_output = keras.layers.LeakyReLU()(x1)

per_network = Model(inputs=fmri_input, outputs=per_output)

per_network.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 6, 14848)]   0                                            
__________________________________________________________________________________________________
BN_0 (BatchNormalization)       (None, 6, 14848)     59392       input_3[0][0]                    
__________________________________________________________________________________________________
Percep_0 (Conv1D)               (None, 6, 128)       5701632     BN_0[0][0]                       
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 6, 128)       0           Percep_0[0][0]                   
______________________________________________________________________________________________

In [23]:
def percep_model(per_input):
    lossModelOutputs = per_network(per_input)
    return lossModelOutputs

In [24]:
LEARNING_RATE = 0.0002
LOSS_FACTOR = 0.002 # 0.002
Per_fac = 0.05  #0.05


def r_loss(y_true, y_pred):
    return K.mean(K.square(y_true - y_pred), axis = [1,2])

def kl_loss(y_true, y_pred):
    kl_loss =  -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis = 1)
    return kl_loss

def per_loss(y_true, y_pred):
#     print("***")
    y_true1 = tf.squeeze(y_true, axis = -1)
    y_pred1 = tf.squeeze(y_pred, axis = -1)
    per_true1 = percep_model(y_true1)
    per_pred1 = percep_model(y_pred1)
    perc_loss = K.mean(K.square(per_true1 - per_pred1), axis=[1,2])
    perc_loss = tf.expand_dims(perc_loss, -1)
#     perc_loss = K.mean(perc_loss)
    return perc_loss

def total_loss(y_true, y_pred):
    return r_loss(y_true, y_pred) + LOSS_FACTOR*kl_loss(y_true, y_pred) + Per_fac*per_loss(y_true, y_pred)
  
adam_optimizer = keras.optimizers.Adam(lr = LEARNING_RATE)

vae.compile(optimizer=adam_optimizer, loss = total_loss, metrics = [r_loss, kl_loss, per_loss])

In [27]:
vae.load_weights('C:/Users/YuNan/Downloads/auto_encoder_try/models/lstm_percep_try.hdf5') 

In [28]:
# per_network.get_layer("Percep_0").trainable = False
# per_network.get_layer("Percep_1").trainable = False
# per_network.get_layer("Percep_2").trainable = False
# per_network.get_layer("Percep_3").trainable = False
# per_network.get_layer("BN_0").trainable = False
per_network.trainable = False
for l in per_network.layers:
    l.trainable = False

In [29]:
# xx = np.ones((4,6,14848,1),dtype = 'float32')
# yy = np.ones((4,6,14848,1), dtype = 'float32')
# # type(xx)
# losss = total_loss(xx,yy)
# losss

In [30]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
es = EarlyStopping(monitor='val_loss', 
                   verbose=1, 
                   patience=6, 
                   min_delta=0.0001, 
                   mode='min')
mc = ModelCheckpoint('.../lstm_percep.hdf5', `
                     monitor='val_loss', 
                     verbose=1, 
                     save_best_only=True, 
                     mode='min')
rlr = ReduceLROnPlateau(monitor='val_loss',
                        mode='min',
                        factor=0.1,
                        patience=6)

In [4]:
epochs = 100
history = vae.fit(train_generator, validation_data=test_generator, epochs = epochs, use_multiprocessing=False, callbacks = [es, mc,rlr])

In [3]:
# Plot history: Categorical crossentropy & Accuracy
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='Categorical crossentropy (training data)')
plt.plot(history.history['val_loss'], label='Categorical crossentropy (validation data)')
# plt.plot(history.history['acc'], label='Accuracy (training data)')
# plt.plot(history.history['val_acc'], label='Accuracy (validation data)')
plt.title('Model performance for 3D MNIST Keras Conv3D example')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")

plt.show()